In [35]:
import tensorflow as tf
import glob
import csv
import time
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [90]:
def get_label(label_fl):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_fl, 'r') as f:
        label1 = f.readlines()
        label1 = [l.strip() for l in label1]
    label2 = {}
    count = 0
    for label in label1:
        label2[label] = count
        count += 1
    return label1, label2


train_path = 'C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/train'
labels_path = 'C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/labels.txt'
test_path = 'C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/test'

filenames = [file for file in glob.glob(train_path + '*/*')]  #get all filenames

labels = [file.split('_')[-1].split('.')[0] for file in filenames ] #get label string name eg. AU1, AU2...

label1, label2 = get_label(labels_path) 

labels = [label2[label] for label in labels if label in label2] #map label  name to id number eg  AU2 -> 2


In [91]:
#Major defines
N_INPUTS  = 32*32*3
N_CLASSES = 7
BATCH_SIZE = 32

In [92]:
"""c = 0
for i in filenames:
    if 'mpg' in i:
        c = c+1
print(c)"""

"c = 0\nfor i in filenames:\n    if 'mpg' in i:\n        c = c+1\nprint(c)"

In [93]:
tf.reset_default_graph()

In [94]:
# Create input dataset and generate batches of data
def dataset_distribution(filenames, labels):
    img_str = tf.read_file(filenames)
    img_d = tf.image.decode_png(img_str, channels=3)
    
    #image augmentation
    float_image = tf.image.per_image_standardization(img_d)
    img_d = tf.reshape(float_image , [-1])
    return img_d, labels

#Train data
data = tf.data.Dataset.from_tensor_slices((filenames[0:1800], labels[0:1800]))
data = data.map(dataset_distribution, num_parallel_calls = 3)
data = data.batch(BATCH_SIZE)
data = data.shuffle(buffer_size=200)    #random number: lower so that buffer filling happens faster
iterator = data.make_initializable_iterator()
train_batch = iterator.get_next()  #next_element is tensor of (img_train, y_train)

# #validation data 
val = tf.data.Dataset.from_tensor_slices((filenames[1800:], labels[1800:]))   #validation batch
val = val.map(dataset_distribution, num_parallel_calls = 3)
val = val.batch(100)
validation_iterator = val.make_initializable_iterator()
validation_batch = validation_iterator.get_next()



#For test images
def _parse_function_test(filenames):
    img_str = tf.read_file(filenames)
    img_d = tf.image.decode_png(img_str, channels=3)
    
    #Reserved for image augmentation
    float_image = tf.image.per_image_standardization(img_d)
    img_d = tf.reshape(float_image , [-1])  #flatten
    
    return img_d

filenames_test = [file_t for file_t in glob.glob(test_path + '*/*')]  
filenames_test.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
lst = []
for i in range(len(filenames_test)):
    splt = filenames_test[i].split("\\")[2].split('.')[0]
    lst.append(splt)

#For test data
data_test = tf.data.Dataset.from_tensor_slices(filenames_test)
data_test = data_test.map(_parse_function_test)
data_test = data_test.batch(311)
test_iterator = data_test.make_initializable_iterator()
test_batch = test_iterator.get_next() 

In [95]:
def conv_model(X, N_CLASSES, reuse, is_training):
    
    with tf.variable_scope('Conv', reuse = reuse): #to reuse weights and biases for testing
        
        input_layer = tf.reshape(X, [-1, 32,32,3])

        conv0 = tf.layers.conv2d(
          inputs=input_layer,
          filters=32,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu)
        
        batchnorm0 = tf.layers.batch_normalization(conv0)
        
        pool0 = tf.layers.max_pooling2d(
            inputs = batchnorm0,
            pool_size = 2,
            strides = 2,
            padding = "same",
            data_format='channels_last')
        
        conv1 = tf.layers.conv2d(
          inputs=pool0,
          filters=64,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu) 
       
        batchnorm1 = tf.layers.batch_normalization(conv1)

        pool1 = tf.layers.max_pooling2d(
            inputs = batchnorm1,
            pool_size = 2,
            strides = 2,
            padding = "same",
            data_format='channels_last')
          
        dropout0 = tf.layers.dropout(
            inputs = pool1,
            rate=0.30,
            training = is_training)
        
        conv2 = tf.layers.conv2d(
            inputs=dropout0,
            filters=128,
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu)

        batchnorm2 = tf.layers.batch_normalization(conv2)
        
        pool2 = tf.layers.max_pooling2d(
            inputs = batchnorm2,
            pool_size = 2,
            strides = 2,
            padding="same",
            data_format='channels_last')
       
        conv3 = tf.layers.conv2d(
          inputs=pool2,
          filters=256,
          kernel_size=[3, 3],
          padding="same",
          activation=tf.nn.relu)
        
        batchnorm3 = tf.layers.batch_normalization(conv3)
        
        pool3 = tf.layers.max_pooling2d(
            inputs = batchnorm3,
            pool_size = 2,
            strides = 2,
            padding="same",
            data_format='channels_last')
        
        dropout1 = tf.layers.dropout(
            inputs = pool3,
            rate=0.25,
            training = is_training)

        flatten = tf.layers.flatten(dropout1)

        dense1 = tf.layers.dense(
            inputs = flatten,
            units = 1024,
            activation= tf.nn.relu)

        dense2 = tf.layers.dense(
            inputs = dense1,
            units = 512,
            activation= tf.nn.relu)

        dropout2 = tf.layers.dropout(
            inputs = dense2,
            rate=0.35,
            training = is_training)
        
        dense3 = tf.layers.dense(
            inputs = dropout2,
            units = N_CLASSES)
        
        if is_training: last_layer = dense3     #using sparse cross entropy so no need to apply softmax here
        else: last_layer = tf.nn.softmax(dense3)   #for inference

        return last_layer

In [96]:
X,y = train_batch
validation_X, validation_Y = validation_batch

global_step = tf.Variable(0, dtype=tf.int32, trainable = False, name='global_step')

out_train = conv_model(X, N_CLASSES, reuse = False, is_training = True)

#Cost function as softmax cross entropy
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits (labels = y, logits = out_train))
learning_rate = tf.placeholder(tf.float32)
optimizer = tf.train.AdamOptimizer().minimize(cost, global_step = global_step)

#Train data accuracy
out_test = conv_model(X, N_CLASSES, reuse=True, is_training=False)
pred = tf.equal(tf.argmax(out_test, 1, output_type=tf.int32), y)
acc_train = tf.reduce_mean(tf.cast(pred, tf.float32))

#Validation data accuracy
val_output = conv_model(validation_X, N_CLASSES, reuse=True, is_training=False)
val_prediction = tf.equal(tf.argmax(val_output, 1, output_type=tf.int32), validation_Y)
acc_val = tf.reduce_mean(tf.cast(val_prediction, tf.float32))

#test data prediction
out_test = conv_model(test_batch, N_CLASSES, reuse=True, is_training=False)
test_pred = tf.argmax(out_test, 1 , output_type=tf.int32)


In [97]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [98]:
out_test

<tf.Tensor 'Conv_3/Softmax:0' shape=(?, 7) dtype=float32>

In [104]:
NUM_EPOCHS = 100
tr_accuracy = []
val_accuracy = []
tr_loss = []
val_loss = []
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)   #initialize variables
    sess.run(iterator.initializer)
    
    
    eCount = 1
        
    while (eCount < NUM_EPOCHS):
        stTime = time.time()
        while True:
            try:
                sess.run(optimizer)
            except tf.errors.OutOfRangeError:
                sess.run(iterator.initializer)
                training_loss, training_acc = sess.run([cost, acc_train])
                tr_accuracy.append(training_acc)
                tr_loss.append(training_loss)
                
                sess.run(validation_iterator.initializer)
                validation_acc = sess.run(acc_val)
                validation_loss = sess.run(cost)
                
                tr_loss.append(validation_acc)
                val_loss.append(validation_loss)
                
                print("Epoch {}    Loss: {:,.4f}    Train Accuracy: {:,.2f}    Val Accuracy: {:,.2f}    Time: {:,.2f}"         
                      .format(eCount, training_loss, training_acc, validation_acc, time.time() - stTime))
                
               
                eCount += 1
                break
                
            except KeyboardInterrupt:
                print ('\nInterrupted at epoch %d' % eCount)
            
                eCount = NUM_EPOCHS + 1
                break
    print ('Done with train')
    #Save the model
    save_path = saver.save(sess, 'C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/FC-model1/model', global_step = global_step )
    print (' saved at %s' % save_path)  
    
    sess.run(test_iterator.initializer)
    test_predictions = sess.run(test_pred)
    pred_csv = open('C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/final_res1'+str(eCount)+'.csv', 'w')
    header = ['id','label']
    with pred_csv:
        writer = csv.writer(pred_csv)
        writer.writerow((header[0], header[1]))
        for count, row in enumerate(range(test_predictions.shape[0])):
            writer.writerow((lst[count], label1[test_predictions[count]]))
        print("Write complete")

Epoch 1    Loss: 2.4648    Train Accuracy: 0.03    Val Accuracy: 0.50    Time: 15.29
Epoch 2    Loss: 0.8137    Train Accuracy: 0.91    Val Accuracy: 0.39    Time: 12.59
Epoch 3    Loss: 0.7286    Train Accuracy: 1.00    Val Accuracy: 0.91    Time: 12.83
Epoch 4    Loss: 0.1000    Train Accuracy: 1.00    Val Accuracy: 1.00    Time: 14.47
Epoch 5    Loss: 0.3808    Train Accuracy: 0.75    Val Accuracy: 1.00    Time: 16.85
Epoch 6    Loss: 0.9556    Train Accuracy: 0.75    Val Accuracy: 0.93    Time: 15.07
Epoch 7    Loss: 1.5986    Train Accuracy: 0.75    Val Accuracy: 0.91    Time: 14.60
Epoch 8    Loss: 0.0011    Train Accuracy: 1.00    Val Accuracy: 0.91    Time: 14.55
Epoch 9    Loss: 0.3663    Train Accuracy: 1.00    Val Accuracy: 0.91    Time: 14.83
Epoch 10    Loss: 0.7782    Train Accuracy: 1.00    Val Accuracy: 0.91    Time: 13.08
Epoch 11    Loss: 0.9177    Train Accuracy: 0.62    Val Accuracy: 1.00    Time: 13.02
Epoch 12    Loss: 0.6586    Train Accuracy: 1.00    Val Accurac

Epoch 97    Loss: 1.0944    Train Accuracy: 0.72    Val Accuracy: 0.99    Time: 13.72
Epoch 98    Loss: 0.0851    Train Accuracy: 1.00    Val Accuracy: 1.00    Time: 2,171.54
Epoch 99    Loss: 0.0000    Train Accuracy: 1.00    Val Accuracy: 1.00    Time: 13.69
Done with train
 saved at C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/FC-model1/model-10947
Write complete


In [111]:
#Final result on test data
import pandas as pd
df = pd.read_csv('C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/final_res1200.csv')
df['id'] = df['id'].str.split('_')
a_count = df[df['id'].str[1]==df['label']].count()[0]
total_test_Sample = 311

In [112]:
c = (a_count/total_test_Sample)*100
c

91.318327974276528